## Simulação

In [5]:
import arbitragem as utils


retorno_pct, retorno_risco, sharpe = utils.simular_estrategia(
    moeda1='BTC',
    moeda2='ETH',
    zscore_compra_e_venda=2.0,
    zscore_encerrar_posicao=0.5,
    stop_loss=3.0,
    cooldown_stop_loss=20,
    janela=60
)

print(f"Retorno percentual: {retorno_pct*100:.2f}%")
print(f"Retorno ajustado ao risco: {retorno_risco:.4f}")
print(f"Índice de Sharpe: {sharpe:.4f}")


As séries BTC-ETH não são cointegradas (p-valor = 0.9788)


TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

## Teste Manual

In [ ]:

# Algoritmo de otimizacao
zscv = 1
zse = 0.5

stop_loss = 0

cooldown = 0
janela = 60

dfcodigo = carregar_dados("ETH", "WBTC")

# Intervalo para teste de cointegração
data_inicial = '2025-06-20'
data_final   = '2025-07-19'
df_filtrado = dfcodigo.loc[data_inicial:data_final]


df_sinalizado = gerar_sinais(df_filtrado, zscv, zse, stop_loss, cooldown, janela) #gera os sinais de compra_venda_encerramento

capital_final, retorno_percentual, capital_series, df_com_retorno_financeiro = simular_retorno_por_trade(df_sinalizado) #simula os retornos

#Valores pra gráfico
df_com_retorno_financeiro['ativo1_valor_esperado'] = df_com_retorno_financeiro['Ativo2'] + df_com_retorno_financeiro['rolling_mean']
df_com_retorno_financeiro['ativo1_desvio_preco'] = df_com_retorno_financeiro['ativo1_valor_esperado'] - df_com_retorno_financeiro['Ativo1']
df_com_retorno_financeiro['ativo1_std_preco'] = df_com_retorno_financeiro['ativo1_desvio_preco'] / df_com_retorno_financeiro['rolling_std']

df_com_retorno_financeiro['ativo2_valor_esperado'] = df_com_retorno_financeiro['Ativo1'] - df_com_retorno_financeiro['rolling_mean']
df_com_retorno_financeiro['ativo2_desvio_preco'] = df_com_retorno_financeiro['ativo2_valor_esperado'] - df_com_retorno_financeiro['Ativo2']
df_com_retorno_financeiro['ativo2_std_preco'] = df_com_retorno_financeiro['ativo2_desvio_preco'] / df_com_retorno_financeiro['rolling_std']


df_com_retorno_financeiro.to_excel('retornos.xlsx', index=False)

print(f"\nRetorno acumulado: {retorno_percentual * 100:.2f}%")
print(f"Capital final: ${capital_final:.2f}")
